In [1]:
#import Python's json Encoder and Decoder
import json
from pprint import pprint

#with open('dengueDump_3.23.16_validated.json') as json_data:
#    d = json.load(json_data)
#    json_data.close()
#    print(d)

# Parsing of json file
    
data = []
with open('dengueDump_3.23.16_validated.json') as f:
    for line in f:
        data.append(json.loads(line))

In [2]:
print(len(data))
#pprint(data[1]['lang'])
# pprint(data[0])

13513


In [3]:
# Extraction of portuguese tweets

pt_tweets = []
for tweet in data:
    # Portuguese Tweets are encoded as 'pt' and spanish tweets are encoded as 'es'
    if tweet['lang'] == 'pt':
        pt_tweets.append(tweet)

len(pt_tweets)

10116

In [4]:
# Extraction of tweets tagged as junk(1) or sickness(3) only

pt13_tweets = []
for tweet in pt_tweets:
    # Identify curator ids of each tweet to refer their tags
    editlen = len(tweet['edits'])
    cidList = []
    for i in range(0,editlen):
        cidList.append(tweet['edits'][i]['curator_id'])
    cidListSet = set(cidList)
    cidList = list(cidListSet)
    cid1 = cidList[0]
    cid2 = cidList[1]
#    cid1 = tweet['edits'][0]['curator_id']
#    cid2 = tweet['edits'][1]['curator_id']
    if (tweet['tags'][cid1] != 2 and tweet['tags'][cid2] != 2):
        pt13_tweets.append(tweet)

len(pt13_tweets)

9965

In [5]:
# Differentiating tweets into those with common tags and those with different tags - easier for manipulation

cmntags = []
difftags = []
zerotag = []
for tweet in pt13_tweets:
    editlen = len(tweet['edits'])
    cidList = []
    for i in range(0,editlen):
        cidList.append(tweet['edits'][i]['curator_id'])
    cidListSet = set(cidList)
    cidList = list(cidListSet)
    cid1 = cidList[0]
    cid2 = cidList[1]
    # Curators do not agree on the annotation of all tweets - clash of tags
    if (tweet['tags'][cid1] == tweet['tags'][cid2]):
        cmntags.append(tweet)
    else:
        difftags.append(tweet)
    
    if ((tweet['tags'][cid1] == 0)|(tweet['tags'][cid2] == 0)):
        zerotag.append(tweet)

print(len(cmntags))
print(len(difftags))
# pprint(zerotag)

7758
2207


In [6]:
count3 = 0
for tweet in cmntags:
    cid = tweet['edits'][0]['curator_id']
    if(tweet['tags'][cid] == 3):
        count3 = count3 + 1
# end
print(count3)

2497


In [7]:
import openpyxl
rec = openpyxl.load_workbook('Tweets_reverse_geolocated_updated.xlsx')
rec.get_sheet_names()
sheet = rec.get_sheet_by_name('Tweets_reverse_geolocated_updat')

region = []
state = []
county = []
mregion = []
for i in range(2, 28126):
    val = [sheet.cell(row = i, column = 7).value]
    region.extend(val)
    val = [sheet.cell(row = i, column = 8).value]
    state.extend(val)
    val = [sheet.cell(row = i, column = 9).value]
    county.extend(val)
    val = [sheet.cell(row = i, column = 10).value]
    mregion.extend(val)

In [8]:
regionSet = set(region)
stateSet = set(state)
countySet = set(county)
mregionSet = set(mregion)

print(len(regionSet))
print(len(stateSet))
print(len(countySet))
print(len(mregionSet))

549
205
2573
462


In [9]:
cfile = open('dengueDump_3.23.16_coordinates.txt', 'r')
print(cfile.readline())
print(cfile.readline())

tweetID,userID,date,lat,lon

408779568778997760,354120599,2013-12-05 21:06:58,25.746,-100.277



In [10]:
cfile.close()

In [11]:
#import regex
import re
import string

# Define a function for pre-processing of tweets
def processTweet(tweet):
    
    # t = "blah #ToKi9llAMockingbird #onemore blah"
    # t1 = re.match(r'#([^\s])', t)
    # t = "#A"
    #Convert to lower case
#     tweet = tweet.lower()
    #Convert hyperlinks to a generic term 'URL'
    tweet = re.sub('((www\.[^\s]+)|(https?:\/\/[^\s]+))','',tweet)
    #Convert @username to USER
    tweet = re.sub('(@[^\s]+)|(@[\s][^\s]+)','',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
#     tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet
#end

# Generate a text file of the original tweets and processed tweets for inspection 
# fp = open('compareTweets.txt', 'w')
# fp1 = open('processedTweets.txt', 'w')

# for tweet in cmntags:
#     text = tweet['t']
#     processedtext = processTweet(text)
#     fp.write(text)
#     fp.write('\t')
#     fp.write(processedtext)
#     fp1.write(processedtext)
#     fp.write('\n')
#     fp1.write('\n')
# #end loop
# fp.close()
# fp1.close()

In [12]:
import nltk
stopwords = nltk.corpus.stopwords.words('portuguese')
# print(stopwords[0:10])
stopwords.append('USER')
stopwords.append('URL')

#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.*)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

#start getfeatureVector
def getFeatureVector(tweet, featureVector):
    #split tweet into words
    words = tweet.split()
    for w in words:
        w = w.lower()
        w = re.sub(r'#([^\s]+)', r'\1', w)
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,*._')
        
#         check if the word starts with an alphabet
#         val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        val = re.search(r"^[a-zA-Z]", w)

#         check if the word contains only numbers
#         valnum = re.search(r"^[0-9][0-9]*$", w)
    
        # Experiment
        u = w.encode('unicode-escape')
        bval = re.search(b'\\U', u)
        #ignore if it is a stop word
        if(w in stopwords or val is None):
#         if(w in stopwords or valnum):
            if (bval is None):
                continue
            else:
                featureVector.append(w)
        else:
            featureVector.append(w.lower())
        regex = re.compile(r'#([^\s]+)')
        
    regex = re.compile(r'#([^\s]+)')
    matchObj = regex.findall(tweet)
#     print(matchObj)
    s = len(matchObj)
    fv = []
    for i in range(0,s):
        word = matchObj[i]
        startInd = len(word)
        stopInd = 0
        for i in range(0,len(word)):
            if (i==(len(word)-1)):
                        stopInd = i + 1
                        if (startInd == len(word)):
                            startInd = 0
                        fv.append(word[startInd:stopInd].lower())
                        continue
            if (word[i].isupper()):
                if (startInd != len(word)):
                    stopInd = i
#                     print(i)
#                     print(word[startInd:stopInd])
                    fv.append(word[startInd:stopInd].lower())
                    startInd = i
                else:
                    if (i != 0):
                        startInd = 0
                        stopInd = i
                        fv.append(word[startInd:stopInd].lower())
                        startInd = i
    
    featureVector.extend(fv)
    return featureVector
#end

In [13]:
for i in range(0,5):
    print(i)

print(round(1.49))
dsize = len(cmntags)
print(2*dsize/5)
d = [1, 9, 0, 5]
t = d[:0]
print(len(t))
print(t)
t.append(d[1:2])
print(t)

0
1
2
3
4
1
3103.2
0
[]
[[9]]


In [14]:
# featureList = []
# tweetset = []
# trainset = cmntags[0:5999]
# for tweet in trainset:
#     tweetFV = []
#     text = tweet['t']
#     cid = tweet['edits'][0]['curator_id']
#     tag = tweet['tags'][cid]
#     processedtext = processTweet(text)
#     tweetFV = getFeatureVector(processedtext, tweetFV)
#     featureList.extend(tweetFV)
#     tweetset.append((tweetFV,tag))

# print(len(featureList))
# featureListSet = set(featureList)
# print(len(featureListSet))
# featureList = list(featureListSet)

In [15]:
# Preparation of test data
data = []
with open('dengueDump_3.23.16_nonvalidated.json') as f:
    for line in f:
        data.append(json.loads(line))

len(data)

14611

In [16]:
pprint(data[15])

{'_id': '452666295931371520',
 'acr': '2011-05-29T23:48:51',
 'cc': 'MX',
 'cr': '2014-04-06T00:37:09',
 'f': 'tw20144517192',
 'flng': 123,
 'flrs': 174,
 'lang': 'es',
 'loc': 'Mexico',
 'p': '7d93122509633720',
 'pln': -99.161,
 'plt': 19.381,
 't': 'Cumple dengue (@ El Viejo Camilo w/ @carro_1986) '
      'http://t.co/jyD9U6w5uD',
 'tln': -99.155,
 'tlt': 19.386,
 'uid': '307695404'}


In [17]:
pt_tweets = []
for tweet in data:
    # Portuguese Tweets are encoded as 'pt' and spanish tweets are encoded as 'es'
    if tweet['lang'] == 'pt':
        pt_tweets.append(tweet)

len(pt_tweets)

10622

In [18]:
#start extract_features
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features
#end

# Extract feature vector for all tweets in one shote
# training_set = nltk.classify.util.apply_features(extract_features, tweetset)

# Train the classifier
# NBClassifier = nltk.NaiveBayesClassifier.train(training_set)

In [19]:
# fp = open('outputNB2.txt', 'w')
# testset = cmntags[6000:]
# acc = 0
# for tweet in testset:
#     fv = []
#     # Test the classifier
#     testTweet = tweet['t']
#     processedTestTweet = processTweet(testTweet)
#     x = NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet,fv)))
#     fp.write(testTweet)
#     fp.write('\t')
#     fp.write(str(x))
#     fp.write('\t')
#     cid = tweet['edits'][0]['curator_id']
#     fp.write(str(tweet['tags'][cid]))
#     fp.write('\n')
#     if (x == tweet['tags'][cid]):
#         acc = acc + 1

# fp.close()
# print(acc)
# acc = acc/1858
# print(acc)

In [20]:
import openpyxl
rec = openpyxl.load_workbook('Tweets_reverse_geolocated_updated.xlsx')

In [21]:
rec.get_sheet_names()
sheet = rec.get_sheet_by_name('Tweets_reverse_geolocated_updat')
sheet['A1'].value

'tweetID'

In [25]:
region = []
state = []
county = []
mregion = []
for i in range(2, 28126):
    val = [sheet.cell(row = i, column = 7).value]
    region.extend(val)
    val = [sheet.cell(row = i, column = 8).value]
    state.extend(val)
    val = [sheet.cell(row = i, column = 9).value]
    county.extend(val)
    val = [sheet.cell(row = i, column = 10).value]
    mregion.extend(val)

In [27]:
regionSet = set(region)
stateSet = set(state)
countySet = set(county)
mregionSet = set(mregion)

# print(len(regionSet))
# print(len(stateSet))
# print(len(countySet))
# print(len(mregionSet))

# print(region)

In [28]:
tweetID = []
cfile = open('dengueDump_3.23.16_coordinates.txt', 'r')
x = (cfile.readline()).split(',')
print(x)
for i in range(0,28126):
    x = (cfile.readline()).split(',')
    tweetID.extend(x[0])


['tweetID', 'userID', 'date', 'lat', 'lon\n']


In [ ]:
# Training & n-fold Cross Validation of Naive Bayes classifier

count = 0
n = 5
dsize = len(cmntags)
finacc = 0
finprec = 0
fintnr = 0
finrecall = 0
flist = open('featureList5.txt','w')
fvtweet = open('fvTweet','w')

for i in range(0,1):
    testset = pt_tweets
    trainset = cmntags
#     ind1 = int(i*dsize/n)
#     ind2 = int((i+1)*dsize/n)
#     testset = cmntags[ind1:ind2]
#     trainset = cmntags[:ind1]
#     trainset.extend(cmntags[ind2:])
    
    featureList = []
    tweetset = []
    
    #Exp
    tagList = []
    
#    trainset = cmntags[0:5999]
    for tweet in trainset:
        tweetFV = []
        text = tweet['t']
        cid = tweet['edits'][0]['curator_id']
        tag = tweet['tags'][cid]
        
        # Experiment
        tagList.append(tag)
        
        processedtext = processTweet(text)
        tweetFV = getFeatureVector(processedtext, tweetFV)
        featureList.extend(tweetFV)
        for i in range(0,len(tweetID)):
            if (tweetID[i] == tweet['_id']):
                tweetFV.extend(region[i])
                tweetFV.extend(state[i])
                tweetFV.extend(county[i])
                tweetFV.extend(mregion[i])
                break
        
        tweetset.append((tweetFV,tag))

    featureList.extend(list(regionSet))
    featureList.extend(list(stateSet))
    featureList.extend(list(countySet))
    featureList.extend(list(mregionSet))
    featureList.extend(list(stateSet))
    print(len(featureList))
    featureListSet = set(featureList)
    print(len(featureListSet))
    featureList = list(featureListSet)
#     flist.write("\n".join(featureList))
    
    tagListSet = set(tagList)
    tagList = list(tagListSet)
    print(tagList)
    print('FVs done')

    # Extract feature vector for all tweets in one shote
    training_set = nltk.classify.util.apply_features(extract_features, tweetset)

    # Train the classifier
    NBClassifier = nltk.NaiveBayesClassifier.train(training_set)
    
    fn = 0
    fp = 0
    tp = 0
    tn = 0

In [31]:
for j in range(0,len(testset)):
    tweet = testset[j]
    fv = []
    # Test the classifier
    testTweet = tweet['t']
    processedTestTweet = processTweet(testTweet)
    x = NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet,fv)))
    if (x == 3):
        count = count + 1
    pt_tweets[j]['ctags'] = x
#         fp.write(testTweet)
#         fp.write('\t')
#         fp.write(str(x))
#         fp.write('\t')
#         cid = tweet['edits'][0]['curator_id']
#         fp.write(str(tweet['tags'][cid]))
#         fp.write('\n')
#         if (x == tweet['tags'][cid]):
#             if (x == 1):
#                 tn = tn + 1
#             else:
#                 tp = tp + 1
#         else:
#             if (x == 1):
#                 fn = fn + 1
#             else:
#                 fp = fp + 1

# fp.close()
#     acc = (tp + tn)/(tp + tn + fp + fn)
#     prec = tp/(tp + fp)
#     recall = tp/(tp + fn)
#     tnr = tn/(tn + fp)
#     print('Accuracy')
#     print(acc)
#     print('Precision')
#     print(prec)
#     print('Recall')
#     print(recall)
#     print('TNR')
#     print(tnr)
#     finacc = finacc + acc
#     finrecall = finrecall + recall
#     finprec = finprec + prec
#     fintnr = fintnr + tnr

# finacc = finacc/n
# finrecall = finrecall/n
# finprec = finprec/n
# fintnr = fintnr/n
# print(finacc)
# print(finprec)
# print(finrecall)
# print(fintnr)

In [33]:
print(len(pt_tweets))
print(count)

10622
4074


In [40]:
data2 = []
with open('tweet_master_data2.json') as f:
    for line in f:
        data2.extend(json.loads(line))

import csv
fp = open('test5.csv', 'w', newline='')
a = csv.writer(fp, delimiter=',')
a.writerow(('Time Stamp', 'Tweet Longitude', 'Tweet Latitude', 'Country', 'Region', 'State', 'County', 'Microregion'))

for k in range(0,len(pt_tweets)):
    tweet = pt_tweets[k]
#     if (tweet['lang'] == 'es')|(tweet['country'] != 'Brasil'):
#         continue
#     editlen = len(tweet['edits'])
#     cidList = []
#     for j in range(0,editlen):
#         cidList.append(tweet['edits'][j]['curator_id'])
#     cidListSet = set(cidList)
#     cidList = list(cidListSet)
#     cid1 = cidList[0]
#     cid2 = cidList[1]
    # Curators do not agree on the annotation of all tweets - clash of tags
#     if (tweet['ctags'] == 1):
#         continue
#         print('Common Tags')
#         if (tweet['tags'][cid1] == 3):
#             print('Sickness Tweet')
#             a.writerow((tweet['cr'], tweet['tln'], tweet['tlt'], tweet['country'], tweet['region'], tweet['state'], tweet['county'], tweet['microregion']))
#     else:
#         print('Different Tags')
    testTweet = tweet['t']
    processedTestTweet = processTweet(testTweet)
    x = NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet,fv)))
    data2[k].update({'ctags':x})
#         print('Classified')
    if (tweet['ctags'] == 3):
        for m in range(0,len(data2)):
            tweet2 = data2[m]
            if (tweet['_id'] == tweet2['_id']):
                if(tweet2['country'] != 'Brasil'):
                    break
                else:
                    a.writerow((tweet2['cr'], tweet2['tln'], tweet2['tlt'], tweet2['country'], tweet2['region'], tweet2['state'], tweet2['county'], tweet2['microregion']))
                    break
            

# end
    

In [36]:
fp.close()

In [ ]:
s = "I\like\my!friends"
print(s.split('!'))
print(featureList)

In [18]:
line = 'Cats #AsYouLikeIt fine'

matchObj = re.match(r'#([^\s]+)', line, re.I)

if matchObj:
    print(matchObj.group())
else:
    print('None')

None


In [36]:
tweet = "Ss #rksa pod"
tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
print(tweet)

f = open("Results-1-FV.txt","w")
# print(featureList, file=f)
f.writelines(["%s\t" % item  for item in featureList])

Ss rksa pod


In [ ]:
# print informative features about the classifier
print(NBClassifier.show_most_informative_features(200))

In [21]:
indList = []
for i in range(0,dsize):
        tweet = cmntags[i]
        cid = tweet['edits'][0]['curator_id']
        tag = tweet['tags'][cid]
        if (tag == 0):
            indList.append(i)

print(len(indList))
print(indList)
            
    

0
[]


In [22]:
print(cmntags[174])

{'_id': '466148566793465856', 'pln': -46.423, 'tags': {'540e300a7673826b17a5604c': 1, '555232868624c82a1c6d2ca3': 1}, 'loc': 'Santo André-SP', 'p': '35e1542602b65f19', 'tln': -46.54, 'flng': 2002, 'lang': 'pt', 'v': True, 'plt': -23.711, 'cr': '2014-05-13T05:30:53', 'tlt': -23.649, 'flrs': 686, 'acr': '2009-07-16T06:21:48', 't': '@Churumelass @TukaScaletti Todos preocupados c a Copa e Reeleição Combate ao transmissor da Dengue nada. E olha que são só ações preventivas', 'edits': [{'new': 1, 'date': '2015-05-14T17:59:47.555000', 'curator_id': '555232868624c82a1c6d2ca3', 'field': 'tags'}, {'new': 1, 'old': 1, 'date': '2015-05-16T13:48:47.248000', 'curator_id': '540e300a7673826b17a5604c', 'field': 'tags'}], 'f': 'tw20145130114', 'uid': '57297022', 'cc': 'BR'}


In [53]:
pprint(data[0])

{'_id': '408779568778997760',
 'acr': '2011-08-13T00:59:17',
 'cc': 'MX',
 'cr': '2013-12-05T21:06:58',
 'edits': [{'curator_id': '540e300a7673826b17a5604c',
            'date': '2015-08-15T21:18:45.875000',
            'field': 'tags',
            'new': 1}],
 'f': 'tw201312520232',
 'flng': 213,
 'flrs': 399,
 'lang': 'es',
 'loc': 'Monterrey Es De Tigres ',
 'p': 'c5c72776b9f0cc0b',
 'pln': -100.273,
 'plt': 25.737,
 't': '@Doravelizg  es el Dengueeeee',
 'tags': {'540e300a7673826b17a5604c': 1},
 'tln': -100.277,
 'tlt': 25.746,
 'uid': '354120599',
 'v': False}


In [24]:
tweet = cmntags[174]
print(len(tweet['edits']))

2


In [76]:
from sklearn import svm

def getSVMFeatureVectorAndLabels(tweets, featureList):
    sortedFeatures = sorted(featureList)
    map = {}
    feature_vector = []
    labels = []
    for t in tweets:
        label = 0
        map = {}
        #Initialize empty map
        for w in sortedFeatures:
            map[w] = 0

        tweet_words = t[0]
        tweet_tag = t[1]
        #Fill the map
        for word in tweet_words:
            #process the word (remove repetitions and punctuations)
            word = replaceTwoOrMore(word)
            word = word.strip('\'"?,.')
            #set map[word] to 1 if word exists
            if word in map:
                map[word] = 1
        #end for loop
        values = list(map.values())
        feature_vector.append(values)
        if(tweet_tag == 1):
            label = 1
        elif(tweet_tag == 3):
            label = 3
        labels.append(label)
    #return the list of feature_vector and labels
    return {'feature_vector' : feature_vector, 'labels': labels}
#end

def getSVMFeatureVector(tweet_words, featureList):
    sortedFeatures = sorted(featureList)
    map = {}
    feature_vector = []
    labels = []
    label = 0
    map = {}
    #Initialize empty map
    for w in sortedFeatures:
        map[w] = 0

    #Fill the map
    for word in tweet_words:
        #process the word (remove repetitions and punctuations)
        word = replaceTwoOrMore(word)
        word = word.strip('\'"?,.')
        #set map[word] to 1 if word exists
        if word in map:
            map[word] = 1
    #end for loop
    values = list(map.values())
    feature_vector.append(values)
    #return the list of feature_vector
    return feature_vector
#end

In [83]:
#SVM
n = 5
dsize = len(cmntags)
finacc = 0
acc = 0

for i in range(0,n):
    testset = []
    trainset = []
    ind1 = int(i*dsize/n)
    ind2 = int((i+1)*dsize/n)
    testset = cmntags[ind1:ind2]
    trainset = cmntags[:ind1]
    trainset.extend(cmntags[ind2:])

    featureList = []
    tweetset = []

    #Exp
    tagList = []

    #    trainset = cmntags[0:5999]
    for tweet in trainset:
        tweetFV = []
        text = tweet['t']
        cid = tweet['edits'][0]['curator_id']
        tag = tweet['tags'][cid]

        # Experiment
        tagList.append(tag)

        processedtext = processTweet(text)
        tweetFV = getFeatureVector(processedtext, tweetFV)
        featureList.extend(tweetFV)
        tweetset.append((tweetFV,tag))

    print(len(featureList))
    featureListSet = set(featureList)
    print(len(featureListSet))
    featureList = list(featureListSet)

    print('Got tweetset')
    result = getSVMFeatureVectorAndLabels(tweetset, featureList)
    print('Got FVs')
    clf = svm.SVC(kernel = 'linear')
    feature_vector = result['feature_vector']
    labels = result['labels']
    clf.fit(feature_vector, labels)
    print('Trained')
    

# acc = 0
# finacc = 0

    for tweet in testset:
        fv = []
        # Test the classifier
        testTweet = tweet['t']
        processedTestTweet = processTweet(testTweet)
        fv = getFeatureVector(processedTestTweet,fv)
        res = getSVMFeatureVector(fv, featureList)
        if (res == []):
            continue
        x = clf.predict(res)

    #         fp.write(testTweet)
    #         fp.write('\t')
    #         fp.write(str(x))
    #         fp.write('\t')
        cid = tweet['edits'][0]['curator_id']
    #         fp.write(str(tweet['tags'][cid]))
    #         fp.write('\n')
        if (x[0] == tweet['tags'][cid]):
            acc = acc + 1

    # fp.close()
    print(acc)
    acc = acc/len(testset)
    print(acc)
    finacc = finacc + acc

finacc = finacc/n
print(finacc)

    
    

37486
5998
Got tweetset
Got FVs
Trained
1259
0.8117343649258543
39395
6413
Got tweetset
Got FVs
Trained
1325.811734364926
0.8542601381217306
39441
6348
Got tweetset
Got FVs
Trained
1295.8542601381218
0.8354959768782217
37901
6020
Got tweetset
Got FVs
Trained
1277.8354959768783
0.8233476133871639
39925
6524
Got tweetset
Got FVs
Trained
1355.823347613387
0.8735975177921309
0.8396871222210203


In [ ]:
print(x)
print(res)

In [35]:
tm = {}
tm['t1'] = 0
tm['t2'] = 1
tm['t3'] = 4
val = list(tm.values())
print(val)

[0, 4, 1]


In [33]:
tweet = testset[2]
text = tweet['t']
# text = "46 people alive \m/"
print(text)
ptt = processTweet(text)
print(ptt)
words = ptt.split()
print(words)
for w in words:
    u = w.encode('unicode-escape')
    print(u)
        #replace two or more with two occurrences
    w = replaceTwoOrMore(w)
    print(w)
        #strip punctuation
    w = w.strip('\'"?,*.')
    print(w)    
#       check if the word starts with an alphabet
    val = re.search(r"^[a-zA-Z]", w)
    print(val)
    val = re.search(b'\\U', u)
    print(val)
    

NameError: name 'testset' is not defined

In [15]:
# Training & n-fold Cross Validation of Max Entropy classifier

n = 5
dsize = len(cmntags)
finacc = 0
fp = open('featureList.txt','w')

for i in range(0,1):
    testset = []
    trainset = cmntags
#     ind1 = int(i*dsize/n)
#     ind2 = int((i+1)*dsize/n)
#     testset = cmntags[ind1:ind2]
#     trainset = cmntags[:ind1]
#     trainset.extend(cmntags[ind2:])
    
    featureList = []
    tweetset = []
    
    #Exp
    tagList = []
    
#    trainset = cmntags[0:5999]
    for tweet in trainset:
        tweetFV = []
        text = tweet['t']
        cid = tweet['edits'][0]['curator_id']
        tag = tweet['tags'][cid]
        
        # Experiment
        tagList.append(tag)
        
        processedtext = processTweet(text)
        tweetFV = getFeatureVector(processedtext, tweetFV)
        fp.write(' '.join(tweetFV))
        featureList.extend(tweetFV)
        tweetset.append((tweetFV,tag))

    print(len(featureList))
    featureListSet = set(featureList)
    print(len(featureListSet))
    featureList = list(featureListSet)
    
    tagListSet = set(tagList)
    tagList = list(tagListSet)
    print(tagList)

    # Extract feature vector for all tweets in one shote
    training_set = nltk.classify.util.apply_features(extract_features, tweetset)

    # Train the classifier
    MaxEntClassifier = nltk.classify.maxent.MaxentClassifier.train(training_set, 'GIS', trace=3, \
    encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 5)
    print('Training done')
    
    acc = 0
    for tweet in testset:
        fv = []
        # Test the classifier
        testTweet = tweet['t']
        processedTestTweet = processTweet(testTweet)
        x = MaxEntClassifier.classify(extract_features(getFeatureVector(processedTestTweet,fv)))
#         fp.write(testTweet)
#         fp.write('\t')
#         fp.write(str(x))
#         fp.write('\t')
        cid = tweet['edits'][0]['curator_id']
#         fp.write(str(tweet['tags'][cid]))
#         fp.write('\n')
        if (x == tweet['tags'][cid]):
            acc = acc + 1

# fp.close()
    print(acc)
    acc = acc/len(testset)
    print(acc)
    finacc = finacc + acc

finacc = finacc/n
print(finacc)


37486
5998
[1, 3]
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.320
             2          -0.57166        0.680
             3          -0.57141        0.680
             4          -0.57119        0.680
         Final          -0.57098        0.680
Training done
1043
0.6724693745970342
0.13449387491940684


In [57]:
for tweet in data:
        tweet = cmntags[i]
        cid1 = tweet['edits'][0]['curator_id']
        cid2 = tweet['edits'][0]['curator_id']
        tag1 = tweet['tags'][cid1]
        tag2 = tweet['tags'][cid1]
        if (tag1 == 0)|(tag2 == 0):
            pprint(tweet)

In [76]:
import re
t = "blah #ToKi9llAMockingbird #onemore blah"
# t1 = re.match(r'#([^\s])', t)
# t = "#A"
regex = re.compile(r'#([^\s]+)')
matchObj = regex.findall(t)
print(matchObj)
s = len(matchObj)
fv = []
for i in range(0,s):
    word = matchObj[i]
    startInd = len(word)
    stopInd = 0
    for i in range(0,len(word)):
        if (i==(len(word)-1)):
                    stopInd = i + 1
                    if (startInd == len(word)):
                        startInd = 0
                    fv.append(word[startInd:stopInd])
                    continue
        if (word[i].isupper()):
            if (startInd != len(word)):
                stopInd = i
                print(i)
                print(word[startInd:stopInd])
                fv.append(word[startInd:stopInd])
                startInd = i
            else:
                if (i != 0):
                    startInd = 0
                    stopInd = i
                    fv.append(word[startInd:stopInd])
                    startInd = i
                
print(fv)
        
        
# matchObj = re.sub(r'#([^\s]+)',r'\1',matchObj)

['ToKi9llAMockingbird', 'onemore']
7
Ki9ll
8
A
['To', 'Ki9ll', 'A', 'Mockingbird', 'onemore']


In [78]:
s = "hhaahahhahahaa"
pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
s = pattern.sub(r"\1\1", s)
print(s)

hhaahahhahahaa


In [41]:

testset = difftags
# trainset = []
# ind1 = int(i*dsize/n)
# ind2 = int((i+1)*dsize/n)
# testset = cmntags[ind1:ind2]
trainset = cmntags
# trainset.extend(cmntags[ind2:])

featureList = []
tweetset = []

#Exp
tagList = []

#    trainset = cmntags[0:5999]
for tweet in trainset:
    tweetFV = []
    text = tweet['t']
    cid = tweet['edits'][0]['curator_id']
    tag = tweet['tags'][cid]

    # Experiment
    tagList.append(tag)

    processedtext = processTweet(text)
    tweetFV = getFeatureVector(processedtext, tweetFV)
    featureList.extend(tweetFV)
    for i in range(0,len(tweetID)):
        if (tweetID[i] == tweet['_id']):
            tweetFV.extend(region[i])
            tweetFV.extend(state[i])
            tweetFV.extend(county[i])
            tweetFV.extend(mregion[i])
            break

    tweetset.append((tweetFV,tag))

featureList.extend(list(regionSet))
featureList.extend(list(stateSet))
featureList.extend(list(countySet))
featureList.extend(list(mregionSet))
featureList.extend(list(stateSet))
print(len(featureList))
featureListSet = set(featureList)
print(len(featureListSet))
featureList = list(featureListSet)
#     flist.write("\n".join(featureList))

tagListSet = set(tagList)
tagList = list(tagListSet)
print(tagList)
print('FVs done')

# Extract feature vector for all tweets in one shote
training_set = nltk.classify.util.apply_features(extract_features, tweetset)

# Train the classifier
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)

fn = 0
fp = 0
tp = 0
tn = 0
for tweet in testset:
    fv = []
    # Test the classifier
    testTweet = tweet['t']
    processedTestTweet = processTweet(testTweet)
    x = NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet,fv)))
    tweet['ctags'] = x

61316
13924
[1, 3]
FVs done


In [49]:
testset = difftags
for tweet in testset:
    fv = []
    # Test the classifier
    testTweet = tweet['t']
    processedTestTweet = processTweet(testTweet)
    x = NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet,fv)))
    tweet['ctags'] = x

In [50]:
count1 = 0
count3 = 0
for tweet in testset:
        if (tweet['ctags'] == 1):
            count1 = count1 + 1
        if (tweet['ctags'] == 3):
            count3 = count3 + 1
print(count1)
print(count3)
print(len(testset))

1101
1106
2207


In [48]:
print(count1)
print(count3)
print(len(testset))
cache_testdata = testset
print(len(cache_testdata))

6548
4074
10622
10622


In [ ]:
data = []
with open('tweet_master_data.json') as f:
    for line in f:
        data.append(json.loads(line))

In [46]:
import csv

# hello = [['Me','You'],['293', '219'],['13','15']]
hello = testset[0:3]
length = len(hello[0])

with open('test1.csv', 'w') as testfile:
    csv_writer = csv.writer(testfile)
#     for y in range(length):
    csv_writer.writerow([x['_id'] for x in hello])
    csv_writer.writerow([x['cc'] for x in hello])


    

In [85]:
import csv
fp = open('test.csv', 'w', newline='')
a = csv.writer(fp, delimiter=',')
a.writerow(('Time Stamp', 'Tweet Longitude', 'Tweet Latitude', 'Country', 'Region', 'State', 'County', 'Microregion'))


83

In [121]:
data2 = []
with open('tweet_master_data.json') as f:
    for line in f:
        data2.extend(json.loads(line))
        
dsize = len(data2)

In [125]:
count3cmn = 0
count3diff = 0
count2cmn = 0
countcmn = 0
for i in range(0,len(data2)):
    tweet = data2[i]
    if (tweet['lang'] == 'es')|(tweet['country'] != 'Brasil'):
        continue
    editlen = len(tweet['edits'])
    cidList = []
    for j in range(0,editlen):
        cidList.append(tweet['edits'][j]['curator_id'])
    cidListSet = set(cidList)
    cidList = list(cidListSet)
    cid1 = cidList[0]
    cid2 = cidList[1]
    # Curators do not agree on the annotation of all tweets - clash of tags
    if (tweet['tags'][cid1] == tweet['tags'][cid2]):
        countcmn = countcmn + 1
        if (tweet['tags'][cid1] == 3):
            count3cmn = count3cmn + 1
        if (tweet['tags'][cid1] == 2):
            count2cmn = count2cmn + 1
#     else:
#         if(tweet['ctags'] == 3):
#             count3diff = count3diff + 1
print(count3cmn)
print(countcmn)
print(count2cmn)
print(i)

2955
9316
23
13512


In [118]:
        
for i in range(0,dsize):
    tweet = data2[i]
    if (tweet['lang'] == 'es')|(tweet['country'] != 'Brasil'):
        continue
    editlen = len(tweet['edits'])
    cidList = []
    for j in range(0,editlen):
        cidList.append(tweet['edits'][j]['curator_id'])
    cidListSet = set(cidList)
    cidList = list(cidListSet)
    cid1 = cidList[0]
    cid2 = cidList[1]
    # Curators do not agree on the annotation of all tweets - clash of tags
    if (tweet['tags'][cid1] == tweet['tags'][cid2]):
        continue
#         print('Common Tags')
#         if (tweet['tags'][cid1] == 3):
#             print('Sickness Tweet')
#             a.writerow((tweet['cr'], tweet['tln'], tweet['tlt'], tweet['country'], tweet['region'], tweet['state'], tweet['county'], tweet['microregion']))
    else:
#         print('Different Tags')
        testTweet = tweet['t']
        processedTestTweet = processTweet(testTweet)
        x = NBClassifier.classify(extract_features(getFeatureVector(processedTestTweet,fv)))
        data2[i].update({'ctags':x})
#         print('Classified')
#         if (x == 3):
#             print('Sickness Tweet')
#             a.writerow((tweet['cr'], tweet['tln'], tweet['tlt'], tweet['country'], tweet['region'], tweet['state'], tweet['county'], tweet['microregion']))       
# end

In [87]:
fp.close()

In [117]:
pprint(data2[6])

{'_id': '309822840889303040',
 'cc': 'BR',
 'country': 'Brasil',
 'county': 'Campo Mourão',
 'cr': '2013-03-07T19:28:16',
 'ctags': 1,
 'edits': [{'curator_id': '540e300a7673826b17a5604c',
            'date': '2015-05-25T21:57:43.353000',
            'field': 'tags',
            'new': 3},
           {'curator_id': '555232868624c82a1c6d2ca3',
            'date': '2015-07-06T17:30:41.279000',
            'field': 'tags',
            'new': 1,
            'old': 1}],
 'f': 'tw201337184414',
 'lang': 'pt',
 'microregion': 'Microrregião Campo Mourão',
 'p': '0ababd6f282787a7',
 'pln': -52.366,
 'plt': -24.126,
 'region': 'Região Sul',
 'state': 'PR',
 't': 'tomara q eu esteja c dengue',
 'tags': {'540e300a7673826b17a5604c': 3, '555232868624c82a1c6d2ca3': 1},
 'tln': -52.379,
 'tlt': -24.05,
 'uid': '109335884',
 'v': True}


In [120]:
count3cmn = 0
count3diff = 0
for i in range(0,len(data2)):
    tweet = data2[i]
    if (tweet['lang'] == 'es')|(tweet['country'] != 'Brasil'):
        continue
    editlen = len(tweet['edits'])
    cidList = []
    for j in range(0,editlen):
        cidList.append(tweet['edits'][j]['curator_id'])
    cidListSet = set(cidList)
    cidList = list(cidListSet)
    cid1 = cidList[0]
    cid2 = cidList[1]
    # Curators do not agree on the annotation of all tweets - clash of tags
    if (tweet['tags'][cid1] == tweet['tags'][cid2]):
        if (tweet['tags'][cid1] == 3):
            count3cmn = count3cmn + 1
    else:
        if(tweet['ctags'] == 3):
            count3diff = count3diff + 1
print(count3cmn)
print(count3diff)
print(i)

2955
0
13512


In [101]:
print(i)

6


In [104]:
pprint(data2[6])

{'_id': '309822840889303040',
 'cc': 'BR',
 'country': 'Brasil',
 'county': 'Campo Mourão',
 'cr': '2013-03-07T19:28:16',
 'edits': [{'curator_id': '540e300a7673826b17a5604c',
            'date': '2015-05-25T21:57:43.353000',
            'field': 'tags',
            'new': 3},
           {'curator_id': '555232868624c82a1c6d2ca3',
            'date': '2015-07-06T17:30:41.279000',
            'field': 'tags',
            'new': 1,
            'old': 1}],
 'f': 'tw201337184414',
 'lang': 'pt',
 'microregion': 'Microrregião Campo Mourão',
 'p': '0ababd6f282787a7',
 'pln': -52.366,
 'plt': -24.126,
 'region': 'Região Sul',
 'state': 'PR',
 't': 'tomara q eu esteja c dengue',
 'tags': {'540e300a7673826b17a5604c': 3, '555232868624c82a1c6d2ca3': 1},
 'tln': -52.379,
 'tlt': -24.05,
 'uid': '109335884',
 'v': True}
